# refine_structure_static
- - -

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Office of Data and Informatics, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2016-09-08

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm) 
 
- - -

## Introduction

The __refine_structure_static__ calculation refines the lattice parameters of an orthogonal system (crystal structure) by calculating the elastic constant using small displacements, and then using the elastic stiffnesses to iterate towards zero pressure. In refining the lattice parameter values, the box dimensions are allowed to relax, but the relative positions of the atoms within the box are held fixed. 

This calculations provides a quick tool for obtaining both the lattice and elastic constants for a given structure.

__Disclaimer__: With this method there is no guarantee that the resulting parameters are for a stable structure. Allowing internal relaxations may result in different values for some structures. Additionally, some transformation paths may be restricted from ocurring, which allows for unstable structures to be refined.

## Demonstration

Library Imports

In [1]:
#Standard Python imports
import os
import shutil
from copy import deepcopy

#http://www.numpy.org/
import numpy as np

#https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

import iprPy

### 1. Generate Data Files

We have embedded a few files to make the Notebook a self-contained calculation.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

Create working directory

In [2]:
working_dir = 'refine_structure_static'

#This prevents recursive directories upon re-running this cell
if os.path.basename(os.getcwd()) == working_dir:
    working_dir = os.getcwd()
else:
    working_dir = os.path.realpath(working_dir)
    
#move to working_dir
try:
    os.chdir(working_dir)
except:
    os.makedirs(working_dir)
    os.chdir(working_dir)

#### 1.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__2012--Jelinek-B--Al-Si-Mg-Cu-Fe__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [3]:
#Create MEAM library and parameter files for the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential
   
#Create MEAM library file "Jelinek_2012_meamf"
f = open('Jelinek_2012_meamf', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")
f.close()

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
f = open('Jelinek_2012_meam.alsimgcufe', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")
f.close()

#### 1.2 Potential Data Model

This generates the interatomic potential data model file associated with the 2012--Jelinek-B--Al-Si-Mg-Cu-Fe potential. Additional potential data models and their associated artifacts can be found in the ipyrPy/reference-libraries/potentials directory.

In [4]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json', 'w')
f.write("""{
    "LAMMPS-potential": {
        "potential": {
            "key": "1515dd80-1984-49de-bc92-d5724059ff56", 
            "id": "2012--Jelinek-B--Al-Si-Mg-Cu-Fe"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "symbol": "AlS",
                "element": "Al", 
                "mass": 26.9815                
            }, 
            {
                "symbol": "SiS",
                "element": "Si", 
                "mass": 28.086                
            }, 
            {
                "symbol": "MgS", 
                "element": "Mg",
                "mass": 24.305                
            }, 
            {
                "symbol": "CuS", 
                "element": "Cu", 
                "mass": 63.54
            }, 
            {
                "symbol": "FeS", 
                "element": "Fe", 
                "mass": 55.847
            }
        ], 
        "pair_style": {
            "type": "meam"
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                }, 
                {
                    "symbolsList": "True"
                }, 
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                }, 
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close()    

#### 1.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc A1 crystal prototype. Additional prototype data models can be found in the ipyrPy/reference-libraries/prototypes directory.

In [5]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
f = open('A1--Cu--fcc.json', 'w')
f.write("""{
    "crystal-prototype": {
        "identifier": {
            "common": "face-centered cubic", 
            "tag": "fcc", 
            "prototype": "Cu", 
            "Pearson-symbol": "cF4", 
            "Strukturbericht": "A1"
        }, 
        "space-group": {
            "number": 225, 
            "Hermann-Maguin": "F m -3 m", 
            "Schoenflies": "O^5_h", 
            "Wykoff": {
                "letter": "a", 
                "multiplicity": 4
            }
        }, 
        "atomic-system": {
            "cell": {
                "cubic": {
                    "a": {
                        "value": 1.0, 
                        "unit": "scaled"
                    }
                }
            }, 
            "atom": [
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.0, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.0, 
                            0.5, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.0, 
                            0.5
                        ], 
                        "unit": "scaled"
                    }
                }, 
                {
                    "component": 1, 
                    "position": {
                        "value": [
                            0.5, 
                            0.5, 
                            0.0
                        ], 
                        "unit": "scaled"
                    }
                }
            ]
        }
    }
}""")
f.close()

### 2. LAMMPS Script Template(s)

This generates any LAMMPS imput template files that are used by the calculation functions. The calculation functions fill in variable terms in the templates to create complete LAMMPS simulation scripts.

1. cij.template -- Statically evaluates the cohesive energy and pressures for a system both as given and with small strains applied. Provides all the necessary information to calculate the elastic constants for the given system.

#### 2.1 cij.template

In [6]:
f = open('cij.template', 'w')
f.write("""#LAMMPS input script that evaluates the Virial pressure of a system both as
#it was given, and with small strains applied.
#This can be used to estimate the 0 K elastic constants of the system.

<atomman_system_info>

<atomman_pair_info>

#Specify strain range and number of points to examine in the range
variable delta equal <delta>
variable steps equal <steps>
variable deltax equal ${delta}/(${steps}-1)

#Specify variables of the initial configuration's dimensions
variable lx0 equal $(lx)
variable ly0 equal $(ly)
variable lz0 equal $(lz)

#Specify the thermo properties to calculate
variable peatom equal pe/atoms
thermo_style custom step lx ly lz yz xz xy pxx pyy pzz pyz pxz pxy v_peatom pe
thermo_modify format float %.13e

#Compute properties for the initial configuration
run 0

#Compute properties for normal x-direction strains
variable aratio equal 1-${delta}/2.+(v_a-1)*${deltax}
variable xmax equal v_aratio*${lx0}
label loopa
variable a loop ${steps}
change_box all x final 0 ${xmax} remap units box
run 0
next a
jump cij.in loopa
change_box all x final 0 ${lx0} remap units box

#Compute properties for normal y-direction strains
variable bratio equal 1-${delta}/2.+(v_b-1)*${deltax}
variable ymax equal v_bratio*${ly0}
label loopb
variable b loop ${steps}
change_box all y final 0 ${ymax} remap units box
run 0
next b
jump cij.in loopb
change_box all y final 0 ${ly0} remap units box

#Compute properties for normal z-direction strains
variable cratio equal 1-${delta}/2.+(v_c-1)*${deltax}
variable zmax equal v_cratio*${lz0}
label loopc
variable c loop ${steps}
change_box all z final 0 ${zmax} remap units box
run 0
next c
jump cij.in loopc
change_box all z final 0 ${lz0} remap units box

change_box all triclinic

#Compute properties for yz shear strains
variable eyz equal (-${delta}/2.+(v_d-1)*${deltax})*${lz0}
label loopd
variable d loop ${steps}
change_box all yz final ${eyz} remap units box
run 0
next d
jump cij.in loopd
change_box all yz final 0 remap units box

#Compute properties for xz shear strains
variable exz equal (-${delta}/2.+(v_e-1)*${deltax})*${lz0}
label loope
variable e loop ${steps}
change_box all xz final ${exz} remap units box
run 0
next e
jump cij.in loope
change_box all xz final 0 remap units box

#Compute properties for xy shear strains
variable exy equal (-${delta}/2.+(v_f-1)*${deltax})*${ly0}
label loopf
variable f loop ${steps}
change_box all xy final ${exy} remap units box
run 0
next f
jump cij.in loopf
change_box all xy final 0 remap units box""")
f.close()

### 3. Python Calculation Function(s)

These are the Python functions that are at the heart of performing the calculation.

1. __quick_a_Cij__: iteratively relaxes an orthorhombic system to a specified pressure state by using the current elastic constants and pressures to adjust the system's dimensions.

2. __calc_cij__: computes the cohesive energy, pressures and elastic constants for a system by calling LAMMPS to run a script based on cij.template.

#### 3.1 quick_a_Cij Function

Iteratively calls calc_cij to refine a system to the specified pressure state.

Arguments:
    
- __lammps_command__ -- command for running LAMMPS.

- __system__ -- atomman.System to refine.

- __potential__ -- atomman.lammps.Potential representation of a LAMMPS implemented potential.

- __symbols__ -- list of element-model symbols for the Potential that correspond to system's atypes.
    
Keyword Arguments:
    
- __mpi_command__ -- MPI command for running LAMMPS in parallel. Default value is None (serial run).  

- __p_xx, p_yy, p_zz__ -- normal pressures to refine the system to. Default values are 0.0. 

- __delta__ -- the strain range to use in calculating the elastic constants. Default value is 1e-5.

- __tol__ -- the relative tolerance criterion for identifying box size convergence. Default value is 1e-10.

- __diverge_scale__ -- identifies a divergent system if x / diverge_scale < x < x * diverge_scale is not True for x = a,b,c. Default value is 3.

Returns a dictionary containing:

- __system_new__ -- an atomman.System that has been refined from system.

- __C__ -- an atomman.ElasticConstants object for system_new.

- __stress__ -- the virial stress state for system_new.

- __ecoh__ -- the cohesive energy for system_new.

This code is directly copied from calc_refine_structure_static.py.

In [7]:
def quick_a_Cij(lammps_command, system, potential, symbols, mpi_command=None, p_xx=0.0, p_yy=0.0, p_zz=0.0, delta = 1e-5, tol=1e-10, diverge_scale=3.):
    """
    Quickly refines static orthorhombic system by evaluating the elastic constants and the virial pressure.
    
    Arguments:
    lammps_command -- directory location for lammps executable
    system -- atomman.System to statically deform and evaluate a,b,c and Cij at a given pressure
    potential -- atomman.lammps.Potential representation of a LAMMPS implemented potential
    symbols -- list of element-model symbols for the Potential that correspond to the System's atypes
    
    Keyword Arguments:
    pxx, pyy, pzz -- tensile pressures to equilibriate to.  Default is 0.0 for all. 
    delta -- the strain range to use in calculating the elastic constants. Default is 1e-5.    
    tol -- the relative tolerance criterion for identifying box size convergence. Default is 1e-10.
    diverge_scale -- identifies a divergent system if x / diverge_scale < x < x * diverge_scale is not True for x = a,b,c.
    """
    
    #initial parameter setup
    converged = False                   #flag for if values have converged
    
    #define boxes for iterating
    system_current = deepcopy(system)       #system with box parameters being evaluated
    system_old = None                     #system with previous box parameters evaluated
    
    for cycle in xrange(100):
        
        #Run LAMMPS and evaluate results based on system_old
        results = calc_cij(lammps_command, system_current, potential, symbols, p_xx, p_yy, p_zz, delta, cycle)
        system_new = results['system_new']
        
        #Test if box has converged to a single size
        if np.allclose(system_new.box.vects, system_current.box.vects, rtol=tol):
            converged = True
            break
        
        #Test if box has converged to two sizes
        elif system_old is not None and np.allclose(system_new.box.vects, system_old.box.vects, rtol=tol):
            #Run LAMMPS Cij script using average between alat0 and alat1
            box = am.Box(a = (system_new.box.a + system_old.box.a) / 2.,
                         b = (system_new.box.b + system_old.box.b) / 2.,
                         c = (system_new.box.c + system_old.box.c) / 2.)
            system_current.box_set(vects=box.vects, scale=True)
            results = calc_cij(lammps_command, system_current, potential, symbols, p_xx, p_yy, p_zz, delta, cycle+1)                 
            
            converged = True
            break
        
        #Test if values have diverged from initial guess
        elif system_new.box.a < system.box.a / diverge_scale or system_new.box.a > system.box.a * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')
        elif system_new.box.b < system.box.b / diverge_scale or system_new.box.b > system.box.b * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')
        elif system_new.box.c < system.box.c / diverge_scale or system_new.box.c > system.box.c * diverge_scale:
            raise RuntimeError('Divergence of box dimensions')  
        elif results['ecoh'] == 0.0:
            raise RuntimeError('Divergence: cohesive energy is 0')
                
        #if not converged or diverged, update system_old and system_current
        else:
            system_old, system_current = system_current, system_new
    
    #Return values if converged
    if converged:        
        return results
    else:
        raise RuntimeError('Failed to converge after 100 cycles')

#### 3.2 calc_cij Function

Uses LAMMPS and the cij.template script to evaluate the elastic constants and stress state of a system. Assuming linear elasticity, the elastic compliance values and the stress state are used to predict a new system size closer to a target stress state.

Arguments:
    
- __lammps_command__ -- command for running LAMMPS.

- __system__ -- atomman.System to refine.

- __potential__ -- atomman.lammps.Potential representation of a LAMMPS implemented potential.

- __symbols__ -- list of element-model symbols for the Potential that correspond to system's atypes.
    
Keyword Arguments:
    
- __mpi_command__ -- MPI command for running LAMMPS in parallel. Default value is None (serial run).  

- __p_xx, p_yy, p_zz__ -- normal pressures to refine the system to. Default values are 0.0. 

- __delta__ -- the strain range to use in calculating the elastic constants. Default value is 1e-5.

- __cycle__ -- the integer number of the current cycle in quick_a_Cij. Used in renaming the log.lammps file created by cij.template to a unique file name. 

Returns a dictionary containing:

- __system_new__ -- an atomman.System created by scaling system to the new box size guess.

- __C__ -- an atomman.ElasticConstants object for system_new.

- __stress__ -- the virial stress state for system_new.

- __ecoh__ -- the cohesive energy for system_new.

This code is directly copied from calc_refine_structure_static.py.

In [8]:
def calc_cij(lammps_command, system, potential, symbols, p_xx=0.0, p_yy=0.0, p_zz=0.0, delta=1e-5, cycle=0):
    """Runs cij_script and returns current Cij, stress, Ecoh, and new system guess."""
    
    #Get lammps units
    lammps_units = lmp.style.unit(potential.units)
    
    #Define lammps variables
    lammps_variables = {}
    lammps_variables['atomman_system_info'] = lmp.sys_gen(units =       potential.units,
                                                          atom_style =  potential.atom_style,
                                                          ucell =       system,
                                                          size =        np.array([[0,3], [0,3], [0,3]]))
    lammps_variables['atomman_pair_info'] = potential.pair_info(symbols)
    lammps_variables['delta'] = delta
    lammps_variables['steps'] = 2
    
    #Write lammps input script
    template_file = 'cij.template'
    lammps_script = 'cij.in'
    with open(template_file) as f:
        template = f.read()
    with open(lammps_script, 'w') as f:
        f.write('\n'.join(iprPy.tools.fill_template(template, lammps_variables, '<', '>')))
    
    #Run lammps 
    output = lmp.run(lammps_command, lammps_script)
    shutil.move('log.lammps', 'cij-'+str(cycle)+'-log.lammps')
    
    #Extract LAMMPS thermo data. Each term ranges i=0-12 where i=0 is undeformed
    #The remaining values are for -/+ strain pairs in the six unique directions
    lx = uc.set_in_units(np.array(output.finds('Lx')), lammps_units['length'])
    ly = uc.set_in_units(np.array(output.finds('Ly')), lammps_units['length'])
    lz = uc.set_in_units(np.array(output.finds('Lz')), lammps_units['length'])
    xy = uc.set_in_units(np.array(output.finds('Xy')), lammps_units['length'])
    xz = uc.set_in_units(np.array(output.finds('Xz')), lammps_units['length'])
    yz = uc.set_in_units(np.array(output.finds('Yz')), lammps_units['length'])
    
    pxx = uc.set_in_units(np.array(output.finds('Pxx')), lammps_units['pressure'])
    pyy = uc.set_in_units(np.array(output.finds('Pyy')), lammps_units['pressure'])
    pzz = uc.set_in_units(np.array(output.finds('Pzz')), lammps_units['pressure'])
    pxy = uc.set_in_units(np.array(output.finds('Pxy')), lammps_units['pressure'])
    pxz = uc.set_in_units(np.array(output.finds('Pxz')), lammps_units['pressure'])
    pyz = uc.set_in_units(np.array(output.finds('Pyz')), lammps_units['pressure'])
    
    pe = uc.set_in_units(np.array(output.finds('peatom')), lammps_units['energy'])
    
    #Set the six non-zero strain values
    strains = np.array([ (lx[2] -  lx[1])  / lx[0],
                         (ly[4] -  ly[3])  / ly[0],
                         (lz[6] -  lz[5])  / lz[0],
                         (yz[8] -  yz[7])  / lz[0],
                         (xz[10] - xz[9])  / lz[0],
                         (xy[12] - xy[11]) / ly[0] ])

    #calculate cij using stress changes associated with each non-zero strain
    cij = np.empty((6,6))
    for i in xrange(6):
        delta_stress = np.array([ pxx[2*i+1]-pxx[2*i+2],
                                  pyy[2*i+1]-pyy[2*i+2],
                                  pzz[2*i+1]-pzz[2*i+2],
                                  pyz[2*i+1]-pyz[2*i+2],
                                  pxz[2*i+1]-pxz[2*i+2],
                                  pxy[2*i+1]-pxy[2*i+2] ])

        cij[i] = delta_stress / strains[i] 
        
    for i in xrange(6):
        for j in xrange(i):
            cij[i,j] = cij[j,i] = (cij[i,j] + cij[j,i]) / 2

    C = am.ElasticConstants(Cij=cij)
    
    if np.allclose(C.Cij, 0.0):
        raise RuntimeError('Divergence of elastic constants to <= 0')
    try:
        S = C.Sij
    except:
        raise RuntimeError('singular C:\n'+str(C.Cij))

    
    #extract the current stress state
    stress = -1 * np.array([[pxx[0], pxy[0], pxz[0]],
                            [pxy[0], pyy[0], pyz[0]],
                            [pxz[0], pyz[0], pzz[0]]])
    
    s_xx = stress[0,0] + p_xx
    s_yy = stress[1,1] + p_yy
    s_zz = stress[2,2] + p_zz
    
    new_a = system.box.a / (S[0,0]*s_xx + S[0,1]*s_yy + S[0,2]*s_zz + 1)
    new_b = system.box.b / (S[1,0]*s_xx + S[1,1]*s_yy + S[1,2]*s_zz + 1)
    new_c = system.box.c / (S[2,0]*s_xx + S[2,1]*s_yy + S[2,2]*s_zz + 1)
    
    if new_a <= 0 or new_b <= 0 or new_c <=0:
        raise RuntimeError('Divergence of box dimensions to <= 0')
    
    newbox = am.Box(a=new_a, b=new_b, c=new_c)
    system_new = deepcopy(system)
    system_new.box_set(vects=newbox.vects, scale=True)
    
    return {'C':C, 'stress':stress, 'ecoh':pe[0], 'system_new':system_new}

### 4. Run Calculation

Specify run parameters

In [9]:
#Commands for LAMMPS and MPI
lammps_command = 'lmp_serial'
mpi_command =    None

#Paths to the potential data model and directory of artifacts
potential_file = '2012--Jelinek-B--Al-Si-Mg-Cu-Fe.json' #Generated above
potential_dir  = ''                                     #Files generated above in working directory

#System load parameters
load_style = 'system_model'
load_file =  'A1--Cu--fcc.json' #Generated above

#Specify symbols combinations to use
symbols = ['AlS']

#Initial lattice parameter guess
a0 = 4.00

Read the interatomic potential data model

In [10]:
with open(potential_file) as f:
    potential = lmp.Potential(f, potential_dir)

Read the prototype system

In [11]:
system = am.load(load_style, load_file)[0]

Resize the system according to the initial lattice parameter guess

In [12]:
system.box_set(a=a0, b=a0, c=a0, scale=True)

Perform quick_a_Cij

In [13]:
results = quick_a_Cij(lammps_command, system, potential, symbols, mpi_command=mpi_command)  

### 5. Display Results

Print the refined lattice parameters

In [14]:
print 'a =', uc.get_in_units(results['system_new'].box.a, 'angstrom'), 'A'
print 'b =', uc.get_in_units(results['system_new'].box.b, 'angstrom'), 'A'
print 'c =', uc.get_in_units(results['system_new'].box.c, 'angstrom'), 'A'

a = 4.04999918098 A
b = 4.04999918098 A
c = 4.04999918098 A


Print the cohesive energy

In [15]:
print 'Ecoh =', uc.get_in_units(results['ecoh'], 'eV'), 'eV'

Ecoh = -3.35300001418 eV


Print the measured stress state

In [16]:
stress = uc.get_in_units(results['stress'], 'GPa')
print 'Stress_xx =', stress[0,0], 'GPa'
print 'Stress_yy =', stress[1,1], 'GPa'
print 'Stress_zz =', stress[2,2], 'GPa'
print 'Stress_xy =', stress[0,1], 'GPa'
print 'Stress_xz =', stress[0,2], 'GPa'
print 'Stress_yz =', stress[1,2], 'GPa'

Stress_xx = -6.74234854894e-08 GPa
Stress_yy = -6.74234853072e-08 GPa
Stress_zz = -6.74234849847e-08 GPa
Stress_xy = 3.6147192209e-17 GPa
Stress_xz = -6.90334775834e-17 GPa
Stress_yz = 2.01521969915e-16 GPa


Print the orthorhombic elastic constants

In [17]:
cij = uc.get_in_units(results['C'].Cij, 'GPa')
print 'C11 =', cij[0,0], 'GPa' 
print 'C22 =', cij[1,1], 'GPa'
print 'C33 =', cij[2,2], 'GPa'
print 'C12 =', cij[0,1], 'GPa'
print 'C13 =', cij[0,2], 'GPa'
print 'C23 =', cij[1,2], 'GPa'
print 'C44 =', cij[3,3], 'GPa'
print 'C55 =', cij[4,4], 'GPa'
print 'C66 =', cij[5,5], 'GPa'

C11 = 110.529294076 GPa
C22 = 110.529294076 GPa
C33 = 110.529294076 GPa
C12 = 60.904408294 GPa
C13 = 60.9044082939 GPa
C23 = 60.9044082939 GPa
C44 = 28.3919826474 GPa
C55 = 28.3919826474 GPa
C66 = 28.3919826474 GPa
